<a href="https://colab.research.google.com/github/groovyy10/body-type-recommendation/blob/main/Know_yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install pandas numpy scikit-learn tensorflow matplotlib streamlit --quiet


In [4]:
import pandas as pd
import numpy as np

# Original dataset with all required body types
data = {
    "Waist": [30, 28, 34, 32, 36, 38, 40, 42, 44, 46],
    "Hips": [40, 36, 42, 38, 44, 48, 50, 52, 54, 56],
    "Shoulders": [38, 34, 40, 36, 42, 44, 46, 48, 50, 52],
    "BodyType": [
        "Hourglass", "Pear", "Hourglass", "Rectangle", "Apple",
        "Men - Hourglass", "Men - Rectangle", "Men - Oval", "Men - Apple", "Men - Pear"
    ],
    "Gender": [
        "Women", "Women", "Women", "Women", "Women",
        "Men", "Men", "Men", "Men", "Men"
    ]
}
df = pd.DataFrame(data)

# Function to generate synthetic data
def generate_synthetic_data(df, n_samples=100):
    synthetic_data = []
    for _, row in df.iterrows():
        for _ in range(n_samples // len(df)):
            synthetic_entry = {
                "Waist": np.clip(row["Waist"] + np.random.randint(-3, 4), 20, 50),
                "Hips": np.clip(row["Hips"] + np.random.randint(-3, 4), 30, 60),
                "Shoulders": np.clip(row["Shoulders"] + np.random.randint(-3, 4), 30, 60),
                "BodyType": row["BodyType"],
                "Gender": row["Gender"],
            }
            # Ensure logical constraints: Hips > Waist and Shoulders > Waist
            if synthetic_entry["Hips"] > synthetic_entry["Waist"] and synthetic_entry["Shoulders"] > synthetic_entry["Waist"]:
                synthetic_data.append(synthetic_entry)
    return pd.DataFrame(synthetic_data)

# Generate synthetic data
synthetic_df = generate_synthetic_data(df, n_samples=200)

# Combine the original dataset with synthetic data
augmented_df = pd.concat([df, synthetic_df], ignore_index=True)

# Add feature engineering: derived ratios
augmented_df["Waist_to_Hip_Ratio"] = augmented_df["Waist"] / augmented_df["Hips"]
augmented_df["Shoulder_to_Waist_Ratio"] = augmented_df["Shoulders"] / augmented_df["Waist"]

# Save the new dataset
augmented_df.to_csv("augmented_with_features_body_data.csv", index=False)

# Print stats for verification
print("Original Dataset Size:", len(df))
print("Synthetic Data Size:", len(synthetic_df))
print("Augmented Dataset Size:", len(augmented_df))
print("\nSample Augmented Data:")
print(augmented_df.head())


Original Dataset Size: 10
Synthetic Data Size: 193
Augmented Dataset Size: 203

Sample Augmented Data:
   Waist  Hips  Shoulders   BodyType Gender  Waist_to_Hip_Ratio  \
0     30    40         38  Hourglass  Women            0.750000   
1     28    36         34       Pear  Women            0.777778   
2     34    42         40  Hourglass  Women            0.809524   
3     32    38         36  Rectangle  Women            0.842105   
4     36    44         42      Apple  Women            0.818182   

   Shoulder_to_Waist_Ratio  
0                 1.266667  
1                 1.214286  
2                 1.176471  
3                 1.125000  
4                 1.166667  


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the augmented dataset
augmented_df = pd.read_csv("augmented_with_features_body_data.csv")

# Define features and labels
X = augmented_df[["Waist", "Hips", "Shoulders", "Waist_to_Hip_Ratio", "Shoulder_to_Waist_Ratio"]]
y = augmented_df["BodyType"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy after Augmentation: {accuracy * 100:.2f}%")


Model Accuracy after Augmentation: 53.66%


In [6]:
!pip install streamlit pyngrok


In [8]:
from google.colab import files
uploaded = files.upload()  # This will allow you to upload files manually


In [9]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import webcolors
import matplotlib.colors as mcolors

# Load the enhanced dataset
df = pd.read_csv("augmented_with_features_body_data.csv")

# Train the improved model
X = df[["Waist", "Hips", "Shoulders", "Waist_to_Hip_Ratio", "Shoulder_to_Waist_Ratio"]]
y = df["BodyType"]
model = DecisionTreeClassifier(random_state=42)
model.fit(X, y)

# Recommendations and shared images
recommendations = {
   "Hourglass": {
        "clothing": ["Fitted tops", "A-line skirts", "High-waisted pants", "Wrap dresses", "Pencil skirts"],
        "shoes": ["Pointed-toe heels", "Classic pumps", "Ankle boots"],
        "accessories": ["Statement belts", "Chunky earrings", "Clutch bags"],
        "image": "images/Hourglass.png",
    },
    "Pear": {
        "clothing": ["Wide-leg trousers", "Peplum tops", "A-line dresses", "Bootcut jeans", "Off-shoulder tops"],
        "shoes": ["Block heels", "Slingback sandals", "Pointed-toe flats"],
        "accessories": ["Long necklaces", "Structured handbags", "Hoop earrings"],
        "image": "images/pear.png",
    },
    "Rectangle": {
        "clothing": ["Ruffled tops", "Layered outfits", "Belts to define the waist", "Flared skirts", "Peplum blouses"],
        "shoes": ["Ballet flats", "Strappy heels", "Sneakers for casual outfits"],
        "accessories": ["Waist belts", "Layered necklaces", "Patterned scarves"],
        "image": "images/Rectangle.png",
    },
    "Apple": {
        "clothing": ["Empire-waist dresses", "V-neck tops", "Flared pants", "A-line skirts", "Flowy blouses"],
        "shoes": ["Platform heels", "Knee-high boots", "Loafers for casual looks"],
        "accessories": ["Long earrings", "Crossbody bags", "Large bracelets"],
        "image": "images/apple.png",
    },
    "Men - Hourglass": {
        "clothing": ["Fitted polo shirts", "Straight-cut jeans", "V-neck sweaters", "Slim-fit blazers", "T-shirts"],
        "shoes": ["Chelsea boots", "Classic sneakers", "Loafers"],
        "accessories": ["Leather belts", "Minimalist watches", "Sunglasses"],
        "image": "images/men-hourglass.jpg",
    },
    "Men - Rectangle": {
        "clothing": ["Layered outfits", "Slim-fit shirts", "Tailored trousers", "Jackets with shoulders", "Vertical stripes"],
        "shoes": ["Derby shoes", "Classic loafers", "Running sneakers"],
        "accessories": ["Bold watches", "Patterned ties", "Messenger bags"],
        "image": "images/men-rectangle.png",
    },
    "Men - Pear": {
        "clothing": ["Dark-colored shirts", "Straight-cut pants", "Blazers worn open", "Monochromatic outfits"],
        "shoes": ["Oxford shoes", "Slip-on loafers", "Desert boots"],
        "accessories": ["Simple watches", "Suspenders", "Medium-sized bags"],
        "image": "images/men-pear.png",
    },
    "Men - Apple": {
        "clothing": ["Empire-waist shirts", "V-neck sweaters", "Dark jeans", "Vertical-striped shirts", "Structured jackets"],
        "shoes": ["Slip-on loafers", "Chelsea boots", "Desert boots"],
        "accessories": ["Leather belts", "Minimalist watches", "Messenger bags"],
        "image": "images/men-apple.png",
    },
}


# Color recommendations based on skin tone
color_recommendations = {
    "Cool": ["Blue", "Lavender", "Rose", "Grey"],
    "Warm": ["Honey", "Olive", "Coral", "Cream"],
    "Neutral": ["Medium Blue", "Jade", "Red", "White"],
}


# Function to get complementary, analogous, and triadic colors
def get_advanced_color_suggestions(base_colors):
    complementary = []
    analogous = []
    triadic = []

    for color in base_colors:
        try:
            base_rgb = mcolors.to_rgb(color)

            # Complementary color (invert RGB values)
            comp_rgb = [1 - val for val in base_rgb]
            complementary.append(mcolors.to_hex(comp_rgb))

            # Analogous colors (shift hue slightly)
            analogous_1 = [min(val + 0.1, 1) for val in base_rgb]
            analogous_2 = [max(val - 0.1, 0) for val in base_rgb]
            analogous.extend([mcolors.to_hex(analogous_1), mcolors.to_hex(analogous_2)])

            # Triadic colors (rotate 120 degrees on the color wheel)
            triadic_1 = [base_rgb[1], base_rgb[2], base_rgb[0]]  # Rotate channels
            triadic_2 = [base_rgb[2], base_rgb[0], base_rgb[1]]
            triadic.extend([mcolors.to_hex(triadic_1), mcolors.to_hex(triadic_2)])
        except ValueError:
            pass

    return {
        "Complementary": complementary,
        "Analogous": analogous,
        "Triadic": triadic,
    }

# Function to plot color preferences
def plot_color_preferences(skin_tone):
    colors = color_recommendations.get(skin_tone, [])
    sizes = [1] * len(colors)
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.pie(
        sizes,
        labels=colors,
        autopct="%1.1f%%",
        startangle=140,
        colors=plt.cm.Paired.colors[:len(colors)],
    )
    ax.set_title(f"Color Preferences for {skin_tone} Skin Tone", fontsize=10)
    st.pyplot(fig)

# Function to plot advanced color charts
def plot_advanced_colors(advanced_colors):
    cols = st.columns(3)
    for idx, (category, colors) in enumerate(advanced_colors.items()):
        with cols[idx]:
            fig, ax = plt.subplots(figsize=(3, 3))
            sizes = [1] * len(colors)
            ax.pie(
                sizes,
                labels=colors,
                autopct="%1.1f%%",
                startangle=140,
                colors=colors,  # Use colors directly
            )
            ax.set_title(category, fontsize=10)
            st.pyplot(fig)

# Function to get recommendations
def recommend_clothing(body_type, gender):
    key = f"Men - {body_type}" if gender == "Men" else body_type
    return recommendations.get(
        key,
        {"clothing": ["No suggestions available"], "shoes": ["No suggestions available"], "accessories": ["No suggestions available"], "image": None},
    )

def recommend_colors(skin_tone):
    return color_recommendations.get(skin_tone, ["No color recommendations available"])


# Streamlit app
st.title("Body Type and Clothing Recommendation")

# Gender selection
gender = st.sidebar.radio("Select Gender:", ["Women", "Men"])

# Body measurements
st.sidebar.header("Enter Your Measurements")
waist = st.sidebar.number_input("Enter your waist size (inches):", min_value=20, max_value=50, step=1)
hips = st.sidebar.number_input("Enter your hip size (inches):", min_value=20, max_value=60, step=1)
shoulders = st.sidebar.number_input("Enter your shoulder size (inches):", min_value=10, max_value=60, step=1)

# Skin tone selection
skin_tone = st.sidebar.selectbox("Select Your Skin Tone:", ["Cool", "Warm", "Neutral"], index=0)

if st.sidebar.button("Get Recommendation"):
    waist_to_hip_ratio = waist / hips
    shoulder_to_waist_ratio = shoulders / waist
    sample = pd.DataFrame([[waist, hips, shoulders, waist_to_hip_ratio, shoulder_to_waist_ratio]],
                          columns=X.columns)
    body_type = model.predict(sample)[0]
    recommendation = recommend_clothing(body_type, gender)
    colors = recommend_colors(skin_tone)

    st.header(f"Predicted Body Type: {body_type} ({gender})")

    if recommendation["image"]:
        st.image(recommendation["image"], caption=f"{body_type} Body Type", use_container_width=True)
    else:
        st.warning("No image available for this body type.")

    st.subheader("Recommended Clothing:")
    for outfit in recommendation["clothing"]:
        st.write(f"- {outfit}")

    st.subheader("Recommended Shoes:")
    for shoe in recommendation["shoes"]:
        st.write(f"- {shoe}")

    st.subheader("Recommended Accessories:")
    for accessory in recommendation["accessories"]:
        st.write(f"- {accessory}")

    st.markdown("### Recommended Colors")
    plot_color_preferences(skin_tone)

    base_colors = color_recommendations.get(skin_tone, [])
    advanced_colors = get_advanced_color_suggestions(base_colors)

    st.markdown("### Advanced Color Suggestions")
    plot_advanced_colors(advanced_colors)

st.markdown("---")

2024-12-12 09:15:39.494 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.707 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-12 09:15:39.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.733 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.741 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-12 09:15:39.752 Session state does not 

DeltaGenerator()

In [10]:


!ngrok config add-authtoken 2q166C48N6aWgy6j1UT1yWt5Y8v_3axFgCVHBSWXwaGbG1fYm


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
from pyngrok import ngrok

# Start the ngrok tunnel to forward Streamlit's port
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://84a5-34-125-99-200.ngrok-free.app" -> "http://localhost:8501"


In [12]:
!streamlit run app.py &





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.99.200:8501

  Stopping...


In [13]:
!pkill -f streamlit


In [14]:
!streamlit cache clear

